In [1]:
from __future__ import division, print_function

import numpy as np

## Utility Function

In [2]:
def atba(a, b):
    return np.dot(np.dot(a.T, b), a)

## Structure Input Data
Example problem from Weaver and Gere

In [3]:
#Weaver & Gere
xy = np.array([[100.0, 75.0], [0.0, 75.0], [200.0, 0.0]])
conn = np.array([[2, 1, 1], [1, 3, 1]])
bc = np.array([[2, 1, 1, 1], [3, 1, 1, 1]])
mprop = np.array([[1.0e4, 10, 1.0e3]])
jtloads = np.array([[1, 0, -10.0, -1000.0]])
memloads = np.array([[1, 0.0, 12.0,200.0, 0.0, 12.0, -200.0],
                     [2, -6.0, 8.0, 250.0, -6.0, 8.0, -250.0]])

## Location Matrix

In [4]:
# Location Matrix. Related to structure
def pf_calclm(n, bc):
    """
    Calculate location matrix containing degree of freedom numbers of each node
    """
    lm = np.zeros((n, 3), dtype=int)
    nd = 0
    ns = bc.shape[0]
    for i in range(ns):
        node = bc[i, 0]
        lm[node-1, 0:3] = bc[i, 1:4]

    for node in range(n):
        for j in range(3):
            if lm[node, j] == 0:
                nd += 1
                lm[node, j] = nd
            else:
                lm[node, j] = 0
    return lm, nd

n = xy.shape[0]
lm, ndof = pf_calclm(n, bc)
print('Number of degrees of freedom = %d' % ndof)
print('Location matrix')
print(lm)

Number of degrees of freedom = 3
Location matrix
[[1 2 3]
 [0 0 0]
 [0 0 0]]


## Member Stiffness Matrix in Member Coordinates

In [5]:
# Stiffness matrix in local coordinate system. Related to element
def pf_stiff(E, A, I, L):
    """
    Calculate stiffness of a plane frame member in local coordinate system
    """
    k = np.zeros((6, 6), dtype=float)
    k[0, 0] = float(E) * A / L
    k[0, 3] = -k[0, 0]

    k[1,1] = (12.0 * E * I) / L**3
    k[1, 2] = (6.0 * E * I) / L**2
    k[1, 4] = -k[1, 1]
    k[1, 5] = k[1, 2]

    k[2, 2] = (4.0 * E * I) / L
    k[2, 4] = -k[1, 2]
    k[2, 5] = k[2, 2] / 2.0

    k[3, 3] = k[0, 0]

    k[4, 4] = k[1, 1]
    k[4, 5] = -k[1, 2]

    k[5, 5] = k[2, 2]

    for i in range(6):
        for j in range(i):
            k[i, j] = k[j, i]

    return k

k = pf_stiff(1, 1, 1, 1)
print('Stiffness Matrix')
print(k)

k = pf_stiff(1e4, 10, 1e3, 125)
print('Stiffness Matrix')
print(k)

Stiffness Matrix
[[  1.   0.   0.  -1.   0.   0.]
 [  0.  12.   6.   0. -12.   6.]
 [  0.   6.   4.   0.  -6.   2.]
 [ -1.   0.   0.   1.   0.   0.]
 [  0. -12.  -6.   0.  12.  -6.]
 [  0.   6.   2.   0.  -6.   4.]]
Stiffness Matrix
[[  8.00000000e+02   0.00000000e+00   0.00000000e+00  -8.00000000e+02
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   6.14400000e+01   3.84000000e+03   0.00000000e+00
   -6.14400000e+01   3.84000000e+03]
 [  0.00000000e+00   3.84000000e+03   3.20000000e+05   0.00000000e+00
   -3.84000000e+03   1.60000000e+05]
 [ -8.00000000e+02   0.00000000e+00   0.00000000e+00   8.00000000e+02
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00  -6.14400000e+01  -3.84000000e+03   0.00000000e+00
    6.14400000e+01  -3.84000000e+03]
 [  0.00000000e+00   3.84000000e+03   1.60000000e+05   0.00000000e+00
   -3.84000000e+03   3.20000000e+05]]


## Rotation Matrix

In [6]:
# Length and direction cosines of a member. Related to element
def pf_calclen(xy1, xy2):
    """Return length and direction cosines of a plane frame member


    Calculate length and direction cosines of a plane frame member
    """
    delta = xy2 - xy1
    L = np.sqrt(np.sum(delta**2))
    dc = delta / L
    return L, dc

# Rotation matrix. Related to element
def pf_calcrot(dc):
    """Return rotation matrix


    Calculate rotation matrix of a plane frame member
    """
    r = np.zeros((6, 6), dtype=float)
    cx = dc[0]
    cy = dc[1]
    r[0, 0] = cx
    r[0, 1] = cy
    r[1, 0] = -cy
    r[1, 1] = cx
    r[2, 2] = 1
    r[3:6, 3:6] = r[0:3, 0:3]
    return r

p1 = np.array([0, 0], dtype=float)
p2 = np.array([3, 4], dtype=float)
L, dc = pf_calclen(p1, p2)
print(L, dc)

r = pf_calcrot(dc)
print(r)

5.0 [ 0.6  0.8]
[[ 0.6  0.8  0.   0.   0.   0. ]
 [-0.8  0.6  0.   0.   0.   0. ]
 [ 0.   0.   1.   0.   0.   0. ]
 [ 0.   0.   0.   0.6  0.8  0. ]
 [ 0.   0.   0.  -0.8  0.6  0. ]
 [ 0.   0.   0.   0.   0.   1. ]]


## Utility Functions

In [7]:
# Utility function to get number of end joints. Related to element
def pf_get_endjts(imem, conn):
    """Return numbers of first and second end of a plane frame member.
       Members are assumed to be numbered starting from 1
    """
    jt1 = conn[imem - 1, 0]
    jt2 = conn[imem - 1, 1]
    return jt1, jt2

# Utility function to get coordinates of end joints. Related to element
def pf_get_endcoord(imem, xy, conn):
    """Return the coordinates of the first and second end of a plane frame member.
       Joints are assumend to be numbered starting from 1."""
    jt1, jt2 = pf_get_endjts(imem, conn)
    xy1 = xy[jt1 - 1, :]
    xy2 = xy[jt2 - 1, :]
    return xy1, xy2

# Utility function to get member properties. Related to element
def pf_get_memprop(imem, xy, conn, mprop):
    """Return the properties of specified plane frame member.
       Members are assumed to be numbered starting from 1."""
    m = conn[imem-1, 2] - 1
    E = mprop[m, 0]
    A = mprop[m, 1]
    I = mprop[m, 2]
    xy1, xy2 = pf_get_endcoord(imem, xy, conn)
    L, dc = pf_calclen(xy1, xy2)
    return E, A, I, L, dc

# Utility function to get dof numbers at end joints. Related to element
def pf_get_dof(imem, conn, lm):
    """Return degree of freedom numbers of the ends of a plane frame member.
       Members and joints are assumed to be numbered starting from 1."""
    jt1, jt2 = pf_get_endjts(imem, conn)
    memdof = np.array([0, 0, 0, 0, 0, 0])
    memdof[0:3] = lm[jt1-1,:]
    memdof[3:6] = lm[jt2-1, :]
    return memdof

nmem = conn.shape[0]
for imem in range(nmem):
    j1, j2 = pf_get_endjts(imem, conn)
    E, A, I, L, dc = pf_get_memprop(imem, xy, conn, mprop)
    print(imem, j1, j2, E, A, I, L, dc)

0 1 3 10000.0 10.0 1000.0 125.0 [ 0.8 -0.6]
1 2 1 10000.0 10.0 1000.0 100.0 [ 1.  0.]


## Member Stiffness Matrix in Structure Coordinates

In [8]:
# Stiffness matrix in structure coordinate system. Related to element
def pf_gstiff(imem, xy, conn, mprop):
    """
    Calculate the stiffness matrix of a plane frame member in strucutre
    coordinate system
    """
    E, A, I, L, dc = pf_get_memprop(imem, xy, conn, mprop)
    r = pf_calcrot(dc)
    k = pf_stiff(E, A, I, L)
    return atba(r, k)

for imem in range(nmem):
    k = pf_gstiff(imem, xy, conn, mprop)
    print('Member', imem)
    print(k)

Member 0
[[    534.1184    -354.5088    2304.        -534.1184     354.5088    2304.    ]
 [   -354.5088     327.3216    3072.         354.5088    -327.3216    3072.    ]
 [   2304.        3072.      320000.       -2304.       -3072.      160000.    ]
 [   -534.1184     354.5088   -2304.         534.1184    -354.5088   -2304.    ]
 [    354.5088    -327.3216   -3072.        -354.5088     327.3216   -3072.    ]
 [   2304.        3072.      160000.       -2304.       -3072.      320000.    ]]
Member 1
[[  1.00000000e+03   0.00000000e+00   0.00000000e+00  -1.00000000e+03
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   1.20000000e+02   6.00000000e+03   0.00000000e+00
   -1.20000000e+02   6.00000000e+03]
 [  0.00000000e+00   6.00000000e+03   4.00000000e+05   0.00000000e+00
   -6.00000000e+03   2.00000000e+05]
 [ -1.00000000e+03   0.00000000e+00   0.00000000e+00   1.00000000e+03
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00  -1.20000000e+02  -6.00000000e+03   0.00000000e+

## Assembling Structure Stiffness Matrix

In [9]:
# Assemble structure stiffness matrix, contribution from one member. Related to structure
def pf_assemssm(imem, xy, conn, mprop, lm, ssm):
    """
    Superpose stiffness matrix of plane frame member on the structure
    stiffness matrix
    """
    K = pf_gstiff(imem, xy, conn, mprop)
    memdof = pf_get_dof(imem, conn, lm)
    for i in range(len(memdof)):
        if memdof[i]:
            for j in range(len(memdof)):
                if memdof[j]:
                    ii = memdof[i] - 1
                    jj = memdof[j] - 1
                    ssm[ii, jj] += K[i, j]
    return ssm

# Assemble structure stiffness matrix, contribution from all members. Related to structure
def pf_ssm(xy, conn, bc, mprop):
    """
    Assemble structure stiffness matrix by superposing stiffness matrices of
    individual members
    """
    n = xy.shape[0]
    lm, ndof = pf_calclm(n, bc)
    nmem = conn.shape[0]
    ssm = np.zeros((ndof, ndof), dtype=float)
    for imem in range(1, nmem+1):
        ssm = pf_assemssm(imem, xy,conn, mprop, lm, ssm)
    return ssm, lm, ndof

ssm, lm, ndof = pf_ssm(xy, conn, bc, mprop)
print('Number of degrees of freedom =', ndof)
print('Location Matrix')
print(lm)
print('Structure Stiffness Matrix')
print(ssm)

Number of degrees of freedom = 3
Location Matrix
[[1 2 3]
 [0 0 0]
 [0 0 0]]
Structure Stiffness Matrix
[[  1.53411840e+03  -3.54508800e+02   2.30400000e+03]
 [ -3.54508800e+02   4.47321600e+02  -2.92800000e+03]
 [  2.30400000e+03  -2.92800000e+03   7.20000000e+05]]


## Assembling Load Vector

In [10]:
# Assemble structure loadvector due to all joint loads. Related to structure
def pf_assem_loadvec_jl(lm, jtloads, P):
    """
    Superpose joint loads on structure load vector
    """
    nloads = jtloads.shape[0]
    for iload in range(nloads):
        jt = int(jtloads[iload, 0])
        jtdof = lm[jt-1, :]
        for j in range(3):
            if jtdof[j]:
                i = jtdof[j] - 1
                P[i] += jtloads[iload, j+1]
    return P

# Assemble structure loadvector due to one member load. Related to structure
def pf_assem_loadvec_ml(iload, xy, conn, lm, memloads, P):
    """
    Superpose equivalent joint loads due to loads applied on members onto
    structure load vector
    """
    imem = memloads[iload-1, 0]
    xy1, xy2 = pf_get_endcoord(imem, xy, conn)
    L, dc = pf_calclen(xy1, xy2)
    r = pf_calcrot(dc)
    ml = memloads[iload-1, 1:7]
    ml = ml.reshape(len(ml), 1)
    am = np.dot(-r.T, ml)
    memdof = pf_get_dof(imem, conn, lm)
    for i in range(6):
        if memdof[i]:
            ii = memdof[i] - 1
            P[ii] += am[i]
    return P

# Assemble structure loadvector due to all loads. Related to structure
def pf_loadvec(xy, conn, jtloads, memloads, ndof, lm):
    """
    Assemble structure load vector due to joint loads and loads applied
    directly on members
    """
    P = np.zeros((ndof,1), dtype=float)
    P = pf_assem_loadvec_jl(lm, jtloads, P)
    nml = memloads.shape[0]
    for iload in range(1, nml+1):
        P = pf_assem_loadvec_ml(iload, xy, conn, lm, memloads, P)
    return P

P = pf_loadvec(xy, conn, jtloads, memloads, ndof, lm)
print('Load Vector')
print(P)

Load Vector
[[  8.88178420e-16]
 [ -3.20000000e+01]
 [ -1.05000000e+03]]


C:\Users\Satish\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Satish\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


## Solve Stiffness Equation for Displacements

In [11]:
x = np.linalg.solve(ssm, P)
print('Displacements')
print(x)

Displacements
[[-0.02026077]
 [-0.09936002]
 [-0.00179756]]


## Putting It All Together

In [12]:
def pf(xy, conn, bc, mprop, jtloads, memloads):
    nodes = xy.shape[0]
    ssm, lm, ndof = pf_ssm(xy, conn, bc, mprop)
    P = pf_loadvec(xy, conn, jtloads, memloads, ndof, lm)
    x = np.linalg.solve(ssm, P)
    return ssm, x, P, lm, ndof

ssm, x, P, lm, ndof = pf(xy, conn, bc, mprop, jtloads, memloads)
print(ssm)
print(P)
print(x)

[[  1.53411840e+03  -3.54508800e+02   2.30400000e+03]
 [ -3.54508800e+02   4.47321600e+02  -2.92800000e+03]
 [  2.30400000e+03  -2.92800000e+03   7.20000000e+05]]
[[  8.88178420e-16]
 [ -3.20000000e+01]
 [ -1.05000000e+03]]
[[-0.02026077]
 [-0.09936002]
 [-0.00179756]]


C:\Users\Satish\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Satish\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


## Member End Forces

In [13]:
# Element end forces in one element. Related to element
def pf_mem_endforces(imem, xy, conn, mprop, memloads, lm, x):
    """
    Calculate member end forces from joint displacements, for one chosen member
    """
    xy1, xy2 = pf_get_endcoord(imem, xy, conn)
    L, dc = pf_calclen(xy1, xy2)
    E, A, I, L, dc = pf_get_memprop(imem, xy, conn, mprop)
    r = pf_calcrot(dc)
    u = np.zeros((6, 1), dtype=float)
    memdof = pf_get_dof(imem, conn, lm)
    for i in range(6):
        if memdof[i]:
            idof = memdof[i]
            u[i] = x[idof-1]
    uu = np.dot(r, u)
    k = pf_stiff(E, A, I, L)
    f = np.zeros((6, 1), dtype=float)
    f = np.dot(k, uu)

    nml = memloads.shape[0]
    for i in range(nml):
        if memloads[i, 0] == imem:
            f += memloads[i, 1:].reshape(6,1)
    return f

for imem in range(1, nmem+1):
    print('Member Number', imem)
    f = pf_mem_endforces(imem, xy, conn, mprop, memloads, lm, x)
    print(f)
    print()

Member Number 1
[[  20.26076865]
 [  13.13782511]
 [ 436.64755273]
 [ -20.26076865]
 [  10.86217489]
 [-322.86504198]]

Member Number 2
[[  28.72591986]
 [  -4.53327872]
 [-677.13495802]
 [ -40.72591986]
 [  20.53327872]
 [-889.52488224]]



## Example 2: Hall and Kabaila

In [14]:
# Hall & Kabaila
xy = np.array([[0.0, 8.0], [8.0, 8.0], [0.0, 4.0], [8.0, 4.0], [0.0, 0.0], [8.0, 0.0]])
conn = np.array([[5, 3, 1], [3, 1, 1], [6, 4, 1], [4, 2, 1], [3, 4, 2], [1, 2, 2]])
bc = np.array([[5, 1, 1, 1], [6, 1, 1, 1]])
mprop = np.array([[20.0e6, 0.1, 0.0008], [20.0e6, 0.15, 0.0030]])
jtloads = np.array([])
memloads = np.array([[6, 0.0, 10.0, 20.0, 0.0, 10.0, -20.0],
                     [5, 0.0, 40.0, 53.3, 0.0, 40.0, -53.3]])

In [15]:
ssm, x, P, lm, ndof = pf(xy, conn, bc, mprop, jtloads, memloads)
print(ssm)
print(P)
print(x)
nmem = conn.shape[0]
for imem in range(1, nmem+1):
    print('Member Number', imem)
    f = pf_mem_endforces(imem, xy, conn, mprop, memloads, lm, x)
    print(f)
    print()

C:\Users\Satish\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Satish\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


[[  378000.          0.       6000.    -375000.          0.          0.
     -3000.          0.       6000.          0.          0.          0.  ]
 [       0.     501406.25     5625.          0.      -1406.25     5625.
         0.    -500000.          0.          0.          0.          0.  ]
 [    6000.       5625.      46000.          0.      -5625.      15000.
     -6000.          0.       8000.          0.          0.          0.  ]
 [ -375000.          0.          0.     378000.          0.       6000.
         0.          0.          0.      -3000.          0.       6000.  ]
 [       0.      -1406.25    -5625.          0.     501406.25    -5625.
         0.          0.          0.          0.    -500000.          0.  ]
 [       0.       5625.      15000.       6000.      -5625.      46000.
         0.          0.          0.      -6000.          0.       8000.  ]
 [   -3000.          0.      -6000.          0.          0.          0.
    381000.          0.          0.    -375000

## Pretty Printing

In [16]:
# Utility functions. Not directly related to DSM
def print_mat(header, k, fmt = "%12.4f"):
    """Print the array with a heading

    Print the array k, preceded by a header string. Each element of k is
    printed using the format string fmt
    """
    m, n = k.shape
    print(header, "Size: %d x %d" % (m, n))
    fmt = fmt.split(' ')
    for i in range(m):
        print("%5d" % (i), end='')
        for j in range(n):
            print(fmt[j] % (k[i, j]), end='')
        print()
    return

print_mat('Node Coordinates', xy, fmt='%12.4f %12.4f')
print()
print_mat('Member Connectivity', conn, fmt='%5d %5d %5d')
print()
print_mat('Zero Boundary Conditions', bc, fmt='%5d %5d %5d %5d')
print()
print_mat('Member Properties', mprop, fmt='%16.4e %12.4f %12.4f')
print()
print_mat('Joint Loads', memloads, fmt='%5d %12.4f %12.4f %12.4f %12.4f %12.4f %12.4f')

Node Coordinates Size: 6 x 2
    0      0.0000      8.0000
    1      8.0000      8.0000
    2      0.0000      4.0000
    3      8.0000      4.0000
    4      0.0000      0.0000
    5      8.0000      0.0000

Member Connectivity Size: 6 x 3
    0    5    3    1
    1    3    1    1
    2    6    4    1
    3    4    2    1
    4    3    4    2
    5    1    2    2

Zero Boundary Conditions Size: 2 x 4
    0    5    1    1    1
    1    6    1    1    1

Member Properties Size: 2 x 3
    0      2.0000e+07      0.1000      0.0008
    1      2.0000e+07      0.1500      0.0030

Joint Loads Size: 2 x 7
    0    6      0.0000     10.0000     20.0000      0.0000     10.0000    -20.0000
    1    5      0.0000     40.0000     53.3000      0.0000     40.0000    -53.3000
